# **FASE DE ENTRENAMIENTO🧑‍💻📊**

In [ ]:
!pip install -r dependencias.txt

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard

In [3]:
mp_holistic = mp.solutions.holistic # inicia el modelo donde podemos seguir los puntos del cuerpo humano, incluyendo pose, mano y caras.
mp_drawing = mp.solutions.drawing_utils # inicializa el dibujado o trazados de los puntos del modelo.
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh

In [ ]:
category = {
    'VOCALES': ['A','E','I','O','U']
}

In [ ]:
actions = np.array([value for values in category.values() for value in values])

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
no_videos = 30
no_frames = 30

## 1. Entrenamiento para modelo HOLISTIC (POSE, MANO Y CARA)

In [ ]:
DATA_PATH = os.path.join('MP_Data_Webcam_Holistic')

In [ ]:
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_videos):
        window = []
        for frame_num in range(no_frames):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            print(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            print(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)

In [ ]:
log_dir = os.path.join('Entrenamientos')
os.makedirs(log_dir)

In [ ]:
# def cargar_modelo():
#     model = None
#     model = Sequential()
#     model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
#     model.add(LSTM(128,return_sequences=True,activation='relu'))
#     model.add(LSTM(64,return_sequences=False,activation='relu'))
#     model.add(Dense(64,activation='relu'))
#     model.add(Dense(32,activation='relu'))
#     model.add(Dense(actions.shape[0],activation='softmax'))
#     return model

In [ ]:
def cargar_modelo():
    model = Sequential()
    model.add(LSTM(128, return_sequences=True,
                   input_shape=(30, 1662)))
    model.add(Dropout(0.3))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))
    return model

In [ ]:
model = None
model = cargar_modelo()
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
log_dir = 'Entrenamientos/train1'
os.makedirs(log_dir)

In [ ]:
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model.fit(X_train,y_train,epochs=100,callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
res = model.predict(X_test)

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
model.save('models/train1.h5')

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix
mcm = multilabel_confusion_matrix(ytrue, yhat)
l = list(set(labels))

ig, axes = plt.subplots(1, len(l), figsize=(15, 5))

for i, (ax, matrix) in enumerate(zip(axes, mcm)):
    sns.heatmap(matrix, annot=True, fmt="d", ax=ax, cmap="Blues")
    ax.set_title(f'Matriz de Confusión para "{actions[l[i]]}"')
    ax.set_xlabel('Predicción')
    ax.set_ylabel('Valor Verdadero')

plt.tight_layout()
plt.show()

## 2. Entrenamiento para modelo (POSE Y MANO)

In [ ]:
DATA_PATH = os.path.join('MP_Data_Webcam_Pose_Manos')

In [ ]:
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_videos):
        window = []
        for frame_num in range(no_frames):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            print(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            print(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)

In [ ]:
log_dir = os.path.join('Entrenamientos')
os.makedirs(log_dir)

In [ ]:
def cargar_modelo():
    model = Sequential()
    model.add(LSTM(128, return_sequences=True,
                   input_shape=(30, 1662)))
    model.add(Dropout(0.3))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))
    return model

In [ ]:
model = None
model = cargar_modelo()
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
log_dir = 'Entrenamientos/train1'
os.makedirs(log_dir)

In [ ]:
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model.fit(X_train,y_train,epochs=100,callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
res = model.predict(X_test)

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
model.save('models/train1.h5')

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix
mcm = multilabel_confusion_matrix(ytrue, yhat)
l = list(set(labels))

ig, axes = plt.subplots(1, len(l), figsize=(15, 5))

for i, (ax, matrix) in enumerate(zip(axes, mcm)):
    sns.heatmap(matrix, annot=True, fmt="d", ax=ax, cmap="Blues")
    ax.set_title(f'Matriz de Confusión para "{actions[l[i]]}"')
    ax.set_xlabel('Predicción')
    ax.set_ylabel('Valor Verdadero')

plt.tight_layout()
plt.show()

## 3. ENTRENAMIENTO PARA MODELO (MANOS)

In [ ]:
DATA_PATH = os.path.join('MP_Data_Webcam_Manos')

In [ ]:
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_videos):
        window = []
        for frame_num in range(no_frames):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            print(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            print(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)

In [ ]:
log_dir = os.path.join('Entrenamientos')
os.makedirs(log_dir)

In [ ]:
def cargar_modelo():
    model = Sequential()
    model.add(LSTM(128, return_sequences=True,
                   input_shape=(30, 1662)))
    model.add(Dropout(0.3))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))
    return model

In [ ]:
model = None
model = cargar_modelo()
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
log_dir = 'Entrenamientos/train1'
os.makedirs(log_dir)

In [ ]:
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model.fit(X_train,y_train,epochs=100,callbacks=[tb_callback])

In [ ]:
model.summary()

In [ ]:
res = model.predict(X_test)

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
model.save('models/train1.h5')

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix
mcm = multilabel_confusion_matrix(ytrue, yhat)
l = list(set(labels))

ig, axes = plt.subplots(1, len(l), figsize=(15, 5))

for i, (ax, matrix) in enumerate(zip(axes, mcm)):
    sns.heatmap(matrix, annot=True, fmt="d", ax=ax, cmap="Blues")
    ax.set_title(f'Matriz de Confusión para "{actions[l[i]]}"')
    ax.set_xlabel('Predicción')
    ax.set_ylabel('Valor Verdadero')

plt.tight_layout()
plt.show()